# Page Fault Visualizer

This notebook enables visualizing the page fault behavior per file.


In [ ]:
%load_ext autoreload
%autoreload 2
from ipywidgets import widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from utilities import load_mappings, extract_faults

In [ ]:
# Replace with custom output directory if necessary
mapped_faults, file_sizes = load_mappings("example/post-ordering")

In [ ]:
# The file to analyze
file_options = sorted(set(m["file_name"] for m in mapped_faults))
default_file = next(
    (f for f in file_options if f.endswith(".vdex")),
    file_options[0] if file_options else None,
)
file_name_widget = widgets.Dropdown(
    options=file_options,
    value=default_file,
    description="File Name:",
    layout=widgets.Layout(padding="0px 0px 200px 0px", width="800px"),
    disabled=False,
)

display(file_name_widget)

# Param: If applicable, the name of file embedded within APK
# This is useful if native libraries are embedded within the APK instead of extracted out
apk_entry_name = None

In [ ]:
faults_major, _, _ = extract_faults(
    file_name_widget.value,
    apk_entry_name,
    file_sizes,
    mapped_faults,
)

faults_all, _, _ = extract_faults(
    file_name_widget.value,
    apk_entry_name,
    file_sizes,
    mapped_faults,
    include_minor=True,
)

time_fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.07,
    subplot_titles=(
        "All Page Fault Offsets (Major + Minor)",
        "Major Page Fault Offsets Over Time",
        "Cumulative Major Page Faults Over Time",
    ),
)

for is_major in (False, True):
    subset = faults_all[faults_all["is_major"] == is_major]
    time_fig.add_trace(
        go.Scatter(
            x=subset["ts"],
            y=subset["offset"],
            mode="markers",
            name="Major" if is_major else "Minor",
            legendgroup="all_faults",
            marker=dict(
                size=6,
                color="#1f77b4" if is_major else "#ff7f0e",
            ),
        ),
        row=1,
        col=1,
    )

time_fig.add_trace(
    go.Scatter(
        x=faults_major["ts"],
        y=faults_major["offset"],
        mode="markers",
        name="Major Faults",
        marker=dict(size=6, color="#1f77b4"),
        showlegend=False,
    ),
    row=2,
    col=1,
)

time_fig.add_trace(
    go.Scatter(
        x=faults_major["ts"],
        y=faults_major.index,
        mode="lines",
        name="Cumulative Major",
        line=dict(color="#1f77b4"),
        showlegend=False,
    ),
    row=3,
    col=1,
)

time_fig.update_layout(height=900, width=1200, showlegend=True)
time_fig.update_xaxes(title_text="Timestamp", row=3, col=1)
time_fig.update_yaxes(title_text="Offset", row=1, col=1)
time_fig.update_yaxes(title_text="Offset", row=2, col=1)
time_fig.update_yaxes(title_text="Cumulative Count", row=3, col=1)
time_fig.show()

index_fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.07,
    subplot_titles=(
        "Page Fault Offsets By Index",
        "Page Fault Offset Differences",
    ),
)

index_fig.add_trace(
    go.Scatter(
        x=faults_all.index,
        y=faults_all["offset"],
        mode="markers",
        marker=dict(size=6, color="#1f77b4"),
        name="Offsets",
        showlegend=False,
    ),
    row=1,
    col=1,
)

index_fig.add_trace(
    go.Scatter(
        x=faults_all.index,
        y=faults_all["offset_diff"],
        mode="lines",
        line=dict(color="#9467bd"),
        name="Offset Difference",
        showlegend=False,
    ),
    row=2,
    col=1,
)

index_fig.update_layout(height=700, width=1200, showlegend=False)
index_fig.update_xaxes(title_text="Page Fault Index", row=2, col=1)
index_fig.update_yaxes(title_text="Offset", row=1, col=1)
index_fig.update_yaxes(title_text="Offset Difference", row=2, col=1)
index_fig.show()